## 3. División del conjunto de datos

In [ ]:
train_set, val_set, test_set = sp.train_val_test_split(df)
print("Longitud del Training Set:", len(train_set))
print("Longitud del Validation Set:", len(val_set))
print("Longitud del Test Set:", len(test_set))

Longitud del Training Set: 577
Longitud del Validation Set: 193
Longitud del Test Set: 193


In [ ]:
X_train, y_train = sp.remove_labels(train_set, "class")
X_val, y_val = sp.remove_labels(val_set, "class")
X_test, y_test = sp.remove_labels(test_set, "class")

## 4. Preparación del conjunto de datos

In [ ]:
num_attribs = list(X_train.select_dtypes(exclude=['object']))
cat_attribs = list(X_train.select_dtypes(include=['object']))

In [ ]:
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("rbst_scaler", RobustScaler())
])

In [ ]:
class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.encoders = {}

    def fit(self, X, y=None):
        for col in self.columns:
            le = LabelEncoder()
            le.fit(X[col])
            self.encoders[col] = le
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            X_copy[col] = self.encoders[col].transform(X[col])
        return X_copy


In [ ]:
cat_pipeline = Pipeline([("oh", OneHotEncoder())])

In [ ]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
X_train_prep = full_pipeline.fit_transform(X_train)
sparse_matrix = csr_matrix(X_train_prep)
X_train_prep = pd.DataFrame.sparse.from_spmatrix(
    sparse_matrix, 
    columns=list(pd.get_dummies(X_train)), 
    index=X_train.index
)

In [ ]:
le = LabelEncoder()
y_train_prep = le.fit_transform(y_train)

## 5. Entrenamiento del algoritmo

In [ ]:
log_reg = LogisticRegression(C=1000)

In [ ]:
log_reg.fit(X_train_prep, y_train)

LogisticRegression(C=1000)

In [ ]:
df = pd.DataFrame(pd.Series(log_reg.coef_[0]))
df.sort_values(by=0, ascending=False)

,0
20,1.629927
6,1.585111
41,1.237159
63,0.970340
25,0.958113
...,...
23,-0.790893
4,-0.846778
19,-0.915041
7,-1.199648
